# Setup Project and Preprocess data

After moving in the terminal to the current directory, I first created a virtual environment using Python 3.11 with `python3.11 -m venv .venv`. 

I then activated it using `source .venv/bin/activate`.

Finally, I run `python3 -m pip install -r requirements.txt` to install all requirements. I keep the `requirements.txt` file up to date with all the packages I use during the course of the exam.

In [1]:
%load_ext autoreload
%autoreload 2

## I/ Loading data

In [2]:
import numpy as np
import pandas as pd
from utils.visualisations_helpers import *
from utils.cleaning_helpers import *

In [3]:
waves = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm']
indresp = {}

for i, wave in enumerate(waves, start=1):
    try:
        indresp[i] = pd.read_stata(f'data/ukhls/{wave}_indresp.dta', convert_categoricals=False)
    except FileNotFoundError:
        print(f'Error: {wave}_indresp.dta not found.')
        continue
    except Exception as e:
        print(f'Error loading {wave}_indresp.dta: {e}')
        continue

# Load cross-wave variables
xwave = pd.read_stata('data/ukhls/xwavedat.dta', convert_categoricals=False)

/var/folders/r4/hzh5vjhs64sfqz3fgg0md4x80000gn/T/ipykernel_68570/1266792573.py:6: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  indresp[i] = pd.read_stata(f'data/ukhls/{wave}_indresp.dta', convert_categoricals=False)


## II/ Filtering columns

In [4]:
variables_of_interest_base = [
    'pidp', 'hidp', 'month', 'istrtdaty', 'indscui_xw',
    'sf12mcs_dv', 'discrim', 'jbstat', 'ff_oprlg', 'age_dv', 
    'hiqual_dv', 'mreason1', 'mreason2', 'mreason3', 'mreason4', 'mreason5',
    'mreason6', 'mreason7', 'mreason97', 'mlstat', 'gor_dv'
]

variables_of_interest_cross_wave = [
    'pidp', 'memorig', 'psu', 'strata', 'sex', 'birthy', 'ethn_dv',
    'bornuk_dv', 'generation', 'yr2uk4'
]

# Dictionary to store the filtered dataset
filtered_indresp = {}

# Filter each dataset in indresp to include only the variables of interest
# Add a column to each wave indicating the presence of that wave
for i, wave in enumerate(waves, start=1):
    variables_of_interest = ['pidp'] + [f'{wave}_{var}' for var in variables_of_interest_base if var != 'pidp']
    if i in indresp:
        filtered_df = indresp[i][[var for var in variables_of_interest if var in indresp[i].columns]].copy()  # Use .copy() to ensure it's a new DataFrame
        filtered_df[f'{wave}_present'] = 1  # Safely assign new column
        filtered_indresp[i] = filtered_df

# Filter xwave to include only the variables of interest
filtered_xwave = xwave[[var for var in variables_of_interest_cross_wave if var in xwave.columns]].copy()  # Use .copy() here as well


## III/ Merging and reshaping dataset

In [5]:
# Now I can merge all waves on the pidp column (keep all columns and fill missing values with NaN)
from functools import reduce

# Merge all waves
merged_indresp = reduce(lambda left, right: pd.merge(left, right, on='pidp', how='outer'), filtered_indresp.values())

# Fill NaN values in the presence columns with 0
for wave in waves:
    merged_indresp[f'{wave}_present'] = merged_indresp[f'{wave}_present'].fillna(0)

# Merge cross-wave variables
merged_indresp = pd.merge(merged_indresp, filtered_xwave, on='pidp', how='left')

In [6]:
# Transform wide dataframe to long (tidy)
# Identify columns to melt (contain underscores but are not in id_vars)
value_vars = [col for col in merged_indresp.columns if col not in variables_of_interest_cross_wave and '_' in col]

# Melt the dataframe
df_long = pd.melt(merged_indresp, 
                  id_vars = variables_of_interest_cross_wave, 
                  value_vars = value_vars, 
                  var_name='wave_variable', value_name='value')

# Extract the first component as 'wave' and the rest as 'variable'
df_long['wave'] = df_long['wave_variable'].str.extract(r'([a-z])_')[0]
df_long['variable'] = df_long['wave_variable'].str.extract(r'^[a-z]_(.*)')[0]

# Pivot the dataframe to keep variables as columns
df_long = df_long.pivot_table(index=variables_of_interest_cross_wave + ['wave'], 
                              columns='variable', 
                              values='value', 
                              aggfunc='first').reset_index()

## IV/ Data cleaning

In [7]:
final_df = (df_long
            .copy()
            .pipe(recode_negative_as_missing)
            .pipe(recode_binary_sex_column)
            .pipe(recode_binary_variables, ['bornuk_dv', 'discrim', 'ff_oprlg'])
            .pipe(recode_ethnicity)
            .pipe(recode_migrant_generation)
            .pipe(recode_education)
            .pipe(recode_labour_force_status)
            .pipe(recode_region))



# Drop columns which were recoded
final_df = final_df.drop(columns=['ethn_dv', 'generation', 'hiqual_dv', 'jbstat', 'gor_dv'])

100%|██████████| 32/32 [00:06<00:00,  5.22it/s]


In [8]:
# ============================
# === Create new variables ===
# ============================
# 1/ TIME SINCE ARRIVAL IN THE UK
# (Time of interview (istrtdaty) - year of arrival in the UK (yr2uk4))
# Set 'yr2uk4' to NaN for UK-born people
final_df.loc[final_df['bornuk_dv'] == 1, 'yr2uk4'] = np.nan

# Calculate years since arrival as an integer
final_df['years_since_arrival'] = (final_df['istrtdaty'] - final_df['yr2uk4']).astype('Int64')

# Set 'years_since_arrival' to NA where either 'istrtdaty' or 'yr2uk4' is NaN
final_df.loc[final_df[['istrtdaty', 'yr2uk4']].isna().any(axis=1), 'years_since_arrival'] = np.nan

# Categorical variable for years since arrival
bins = [0, 4, 9, 14, 19, np.inf]
labels = ['0-4', '5-9', '10-14', '15-19', '20+']
final_df['years_since_arrival_cat'] = pd.cut(final_df['years_since_arrival'], bins=bins, labels=labels, right=True)

# Binary variable for years since arrival (10+ years)
final_df['years_since_arrival_binary'] = final_df['years_since_arrival'].apply(lambda x: 1 if pd.notna(x) and x > 10 else (0 if pd.notna(x) else pd.NA))

# --------------------------------
# 2/ IMMIGRANT VARIABLES 
# Binary (bornuk_dv == 0)
final_df = create_immigrant_variable(final_df)

# Categorical
final_df['imm_group'] = final_df['bornuk_dv'].map({1: 'UK-born', 0: 'Immigrants'})

# --------------------------------
# 3/ UNEMPLOYMENT
final_df['unemployed'] = final_df['jbstat_recoded'].apply(lambda x: 1 if x == 'Unemployed' else 0)

In [9]:
# 4/ REASON FOR MIGRATION
# !! only meaningful for immigrants, so requires the intervention of an indicator variable !!
# Note: this is an exception to the general rule that terms should not be included as interactions without a main effect term

# Mapping of reasons to categories
mreason_labels = {
    'mreason1': 'Work/Education',
    'mreason5': 'Work/Education',
    'mreason2': 'Family',
    'mreason3': 'Family',
    'mreason4': 'Family',
    'mreason6': 'Political safety',
    'mreason7': 'Wanted to live in UK',
    'mreason97': 'Other',
}

# Ensure the necessary columns are in the DataFrame
required_columns = list(mreason_labels.keys())
missing_columns = [col for col in required_columns if col not in final_df.columns]
if missing_columns:
    raise ValueError(f"Missing required columns in final_df: {missing_columns}")

# Convert the mapping dictionary to a DataFrame
mreason_mapping = pd.DataFrame(
    list(mreason_labels.items()), columns=['Variable', 'Category']
)

# Reshape the DataFrame to long format for processing reasons
melted = (
    final_df[required_columns]
    .reset_index()
    .melt(id_vars='index', var_name='Variable', value_name='Value')
    .query('Value == 1')  # Filter only selected reasons
    .merge(mreason_mapping, on='Variable', how='left')  # Map reasons to categories
)

# Resolve multiple reasons by selecting the first category per respondent
mreason_by_index = (
    melted.groupby('index')['Category']
    .first()  # Use the first match if multiple reasons exist
    .reindex(final_df.index, fill_value='Unknown')  # Fill 'Unknown' for respondents with no reason
)

# Add the 'mreason' column to the original DataFrame
final_df['mreason'] = mreason_by_index

# Assign 'N/A' to UK-born individuals
final_df.loc[final_df['imm_group'] == 'Domestic-born', 'mreason'] = 'N/A'

# Drop the original migration reason columns
final_df = final_df.drop(columns=required_columns)

# Ensure final column updates are meaningful
final_df['mreason'] = final_df['mreason'].fillna('Unknown')  # Handle any unexpected missing values

# Ensure 'mreason' is a categorical variable with 'Unknown' as the reference category
categories_order = ['Unknown', 'Work/Education', 'Family', 'Political safety', 'Wanted to live in UK', 'Other', 'N/A']
final_df['mreason'] = pd.Categorical(
    final_df['mreason'],
    categories=categories_order,
    ordered=True  # Makes it an ordered categorical variable
)

In [10]:
# Filter the dataframe to include only the discrimination variable and related columns
discrimination_df = final_df[['pidp', 'discrim']].dropna(subset=['discrim'])

## V/ Choice of wave(s)

In [11]:
# Now that all variables of interest have been filtered and cleaned
# I can choose which wave(s) to use for the analysis: this will depend on missing values 

# Initial condition: Filter final_df to include only rows with sf12mcs_dv, immigrant group and ethnicity
data = final_df.dropna(subset=['sf12mcs_dv', 'imm_group', 'ethn_dv_recoded'])

# Which waves include known data on reasons for migration
# Filter dataframe to include only rows with reasons for migration different than 'Unknown' and 'N/A'
known_migration_df = data[(data['mreason'] != 'Unknown') & (data['mreason'] != 'N/A')]
print(known_migration_df['wave'].value_counts())
print(known_migration_df['mreason'].unique())

wave
g    3605
Name: count, dtype: int64
['Work/Education', 'Family', 'Wanted to live in UK', 'Other', 'Political safety']
Categories (7, object): ['Unknown' < 'Work/Education' < 'Family' < 'Political safety' < 'Wanted to live in UK' < 'Other' < 'N/A']


The only wave which includes observations with both mental health of immigrants and their reason to migrate is Wave 7.

## VI/ Creation of the analysis datasets (wave 7)

In [17]:
# ============================
# === Create final dataset ===
# ============================
# Cross-sectional analysis: wave 7
df_wave_7 = (final_df
             .query("wave == 'g' and present == 1")
             .dropna(axis=1, how='all')
             .dropna(subset=['bornuk_dv', 'sf12mcs_dv', 'sex', 'ethn_dv_recoded', 'hiqual_dv_recoded', 'unemployed', 'gor_dv_recoded'])
             .query("16 <= age_dv <= 80")
             .query("(imm_group == 'Immigrants') or generation_recoded != '2nd generation'")
             .merge(discrimination_df, on='pidp', how='left')
            )

# Pickle the final dataset
df_wave_7.to_pickle('data/wave7_data.pkl')

# Save the final dataset to a CSV file (does not preserve the categorical data types)
df_wave_7.to_csv('data/wave7_data.csv', index=False)

In [18]:
# ======================
# === Part 1 Dataset ===
# ======================
# Remove immigrants who have lived in the UK for more than 15 years
df_wave_7_part1 = df_wave_7.query("imm_group == 'UK-born' or years_since_arrival_binary == 0")

# Pickle the final dataset
df_wave_7_part1.to_pickle('data/wave7_data_part1.pkl')

# Save the final dataset to a CSV file (does not preserve the categorical data types)
df_wave_7_part1.to_csv('data/wave7_data_part1.csv', index=False)

In [19]:
# ======================
# === Part 2 Dataset ===
# ======================
# Reduce sample to only immigrants
df_wave_7_part2 = df_wave_7[df_wave_7['imm_group'] == 'Immigrants']

# Remove immigrants without years since arrival
df_wave_7_part2 = df_wave_7_part2.dropna(subset=['years_since_arrival'])

# Pickle the final dataset
df_wave_7_part2.to_pickle('data/wave7_data_part2.pkl')

# Save the final dataset to a CSV file (does not preserve the categorical data types)
df_wave_7_part2.to_csv('data/wave7_data_part2.csv', index=False)